In [2]:
import pyblp
import numpy as np
import pandas as pd

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'0.8.1'

In [32]:
data = pd.read_csv("psetOne.csv")

In [24]:
product_data = data.rename({'price': 'prices', 'Market': 'market_ids', 'Shares': 'shares'}, axis='columns')
product_data.columns = data1.columns.str.replace('z','demand_instruments')

In [25]:
product_data.head()

,intercept,prices,x3,x4,x5,demand_instruments1,demand_instruments2,demand_instruments3,demand_instruments4,demand_instruments5,...,demand_instruments13,demand_instruments14,demand_instruments15,demand_instruments16,demand_instruments17,demand_instruments18,demand_instruments19,demand_instruments20,market_ids,shares
0,1,12.108455,0.160992,4.812402,1.782189,1,0.378256,0.488768,0.102250,0.957012,...,0.568233,0.915644,0.747243,0.105835,0.219022,0.817923,0.580199,0.447471,1,0.010879
1,1,13.181407,1.485405,4.801389,1.934532,1,0.604463,0.815648,0.714064,0.195970,...,0.986930,0.558251,0.263951,0.141410,0.343043,0.391999,0.435976,0.608725,1,0.040582
2,1,15.491495,1.503773,4.771385,1.641470,1,0.470545,0.862062,0.867709,0.789975,...,0.406572,0.513993,0.207370,0.997862,0.345570,0.914552,0.941882,0.704574,1,0.003859
3,1,11.178702,4.798550,4.610094,1.844118,1,0.422738,0.362674,0.373128,0.025455,...,0.915208,0.151043,0.852034,0.843238,0.209150,0.672822,0.362544,0.478866,1,0.761173
4,1,13.009283,0.939336,4.598974,1.730790,1,0.840465,0.161859,0.476482,0.596515,...,0.203302,0.507820,0.315017,0.746393,0.656801,0.668211,0.958055,0.047648,1,0.014363


In [72]:
X1_formulation = pyblp.Formulation('0 + prices+x3 + x4 + x5 + demand_instruments2 + demand_instruments3 + demand_instruments4 + demand_instruments5 + demand_instruments6 + demand_instruments7 + demand_instruments8 + demand_instruments9 + demand_instruments10 + demand_instruments11 + demand_instruments12 + demand_instruments13 + demand_instruments14 + demand_instruments15 + demand_instruments16 + demand_instruments17 + demand_instruments18 + demand_instruments19 + demand_instruments20')
#X1_formulation = pyblp.Formulation('0 + prices', absorb='C(market_ids)')
#X1_formulation = pyblp.Formulation('0 + prices')
X2_formulation = pyblp.Formulation('1 + prices + x3 + x4 + x5')
product_formulations = (X1_formulation, X2_formulation)
product_formulations

(prices + x3 + x4 + x5 + demand_instruments2 + demand_instruments3 + demand_instruments4 + demand_instruments5 + demand_instruments6 + demand_instruments7 + demand_instruments8 + demand_instruments9 + demand_instruments10 + demand_instruments11 + demand_instruments12 + demand_instruments13 + demand_instruments14 + demand_instruments15 + demand_instruments16 + demand_instruments17 + demand_instruments18 + demand_instruments19 + demand_instruments20,
 1 + prices + x3 + x4 + x5)

In [68]:
mc_integration = pyblp.Integration('monte_carlo', size=20, seed=0)
mc_integration

Configured to construct nodes and weights with Monte Carlo simulation.

In [69]:
pr_integration = pyblp.Integration('product', size=5)
pr_integration

Configured to construct nodes and weights according to the level-5 Gauss-Hermite product rule.

In [70]:
mc_problem = pyblp.Problem(product_formulations, product_data, integration=mc_integration)
#mc_problem.MD = 20
mc_problem

Dimensions:
 T    N    I     K1    K2 
---  ---  ----  ----  ----
100  367  2000   1     5  

Formulations:
       Column Indices:           0       1      2    3    4 
-----------------------------  ------  ------  ---  ---  ---
 X1: Linear Characteristics    prices                       
X2: Nonlinear Characteristics    1     prices  x3   x4   x5 

In [62]:
instrument_formulation = pyblp.Formulation('demand_instruments1 + demand_instruments2 + demand_instruments3 + demand_instruments4 + demand_instruments5 + demand_instruments6 + demand_instruments7 + demand_instruments8 + demand_instruments9 + demand_instruments10 + demand_instruments11 + demand_instruments12 + demand_instruments13 + demand_instruments14 + demand_instruments15 + demand_instruments16 + demand_instruments17 + demand_instruments18 + demand_instruments19 + demand_instruments20')
local_instruments = pyblp.build_differentiation_instruments(instrument_formulation , product_data)
local_instruments.shape

KeyError: 'product_data must have market_ids and firm_ids fields.'

In [63]:
pr_problem = pyblp.Problem(product_formulations, product_data, integration=pr_integration)
pr_problem.MD = 20
pr_problem

Dimensions:
 T    N     I      K1    K2    MD    ED 
---  ---  ------  ----  ----  ----  ----
100  367  312500   23    5     20    1  

Formulations:
       Column Indices:           0       1      2    3            4                    5                    6                    7                    8                    9                   10                   11                    12                    13                    14                    15                    16                    17                    18                    19                    20                    21                    22         
-----------------------------  ------  ------  ---  ---  -------------------  -------------------  -------------------  -------------------  -------------------  -------------------  -------------------  -------------------  --------------------  --------------------  --------------------  --------------------  --------------------  --------------------  --------------------  -----

In [64]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-10})
bfgs

Configured to optimize using the BFGS algorithm implemented in SciPy with analytic gradients and options {gtol: +1.0E-10}.

In [73]:
results1 = mc_problem.solve(sigma=np.ones((5, 5)), optimization=bfgs, method ='1s')
results1

LinAlgError: cond is not defined on empty arrays

In [ ]:
results2 = pr_problem.solve(sigma=np.ones((5, 5)), optimization=bfgs)
results2

In [149]:
#logit formulation test
logit_formulation = pyblp.Formulation('prices + x3 +x4 + x5 + demand_instruments2', absorb='C(market_ids)')
logit_formulation

prices + x3 + x4 + x5 + demand_instruments2 + Absorb[C(market_ids)]

In [150]:
log_problem = pyblp.Problem(logit_formulation, data1)

In [151]:
log_problem

Dimensions:
 T    N    K1    MD    ED 
---  ---  ----  ----  ----
100  367   5     4     1  

Formulations:
     Column Indices:          0      1    2    3            4         
--------------------------  ------  ---  ---  ---  -------------------
X1: Linear Characteristics  prices  x3   x4   x5   demand_instruments2

In [228]:
#BLP type with no correlation
#introduce bounds on sigma guess
sigma=np.diag([1,1,1,1,1])
sigma

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]])

In [229]:
sigma_bounds = (np.zeros_like(sigma), np.diag([100,100,100,100,100]))
sigma_bounds

(array([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), array([[100,   0,   0,   0,   0],
        [  0, 100,   0,   0,   0],
        [  0,   0, 100,   0,   0],
        [  0,   0,   0, 100,   0],
        [  0,   0,   0,   0, 100]]))

In [230]:
results3 = mc_problem.solve(sigma=sigma,sigma_bounds=sigma_bounds, optimization=bfgs)
results3

Problem Results Summary:
GMM   Objective  Gradient      Hessian         Hessian     Weighting Matrix  Covariance Matrix
Step    Value      Norm    Min Eigenvalue  Max Eigenvalue  Condition Number  Condition Number 
----  ---------  --------  --------------  --------------  ----------------  -----------------
 2    +1.2E-29   +4.8E-15     -4.7E-08        +6.5E-08         +7.4E+02          +5.5E+18     

Cumulative Statistics:
Computation  Optimization   Objective   Fixed Point  Contraction
   Time       Iterations   Evaluations  Iterations   Evaluations
-----------  ------------  -----------  -----------  -----------
 00:00:20         0             4          2892         8869    

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:      1         prices        x3          x4          x5    
------  ----------  ----------  ----------  ----------  ----------
  1      +1.0E+00                                                 
        (+1.8E-02)                              